In [1]:
import numpy as np
import pandas as pd

>I manually downloaded the CIFAR-10 dataset from the [official website](https://www.cs.toronto.edu/~kriz/cifar.html) and loaded the data , but you can use the tensorflow built-in CIFAR-10 dataset.

## Data preparation ##

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [3]:
all_data = []
all_labels = []

batch_files = ['data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5']

for batch in batch_files:
    batch_data = unpickle(f'cifar-10-batches/{batch}')
    
    all_data.append(batch_data[b'data'])  # Image data
    all_labels.extend(batch_data[b'labels'])  # Corresponding labels

all_data = np.concatenate(all_data)
X_train = all_data.reshape(len(all_data),3,32,32).transpose(0,2,3,1)
y_train = np.array(all_labels)
test_batch = unpickle('cifar-10-batches/test_batch')
X_test = test_batch[b'data']
X_test = X_test.reshape(len(X_test),3,32,32).transpose(0,2,3,1)
y_test = test_batch[b'labels']

In [4]:
np.shape(X_train)

(50000, 32, 32, 3)

In [5]:
np.shape(all_labels)

(50000,)

In [6]:
#normalize the data
X_train = X_train/255
X_test = X_test/255
#one hot encode the labels
y_train,y_test = pd.get_dummies(y_train),pd.get_dummies(y_test)
y_train,y_test = y_train.astype("int"),y_test.astype("int")
y_train,y_test = y_train.to_numpy(),y_test.to_numpy()

In [7]:
np.shape(y_train)

(50000, 10)

## Model Training ##

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from sklearn.metrics import accuracy_score

In [9]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', input_shape=(32,32,3)),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(128, activation='relu', kernel_initializer='he_normal'),
    Dense(10, activation='softmax')  # Output layer for 10 classes
])

c:\Users\dhiab\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [11]:
model.fit(X_train,y_train,epochs=25,batch_size=128)

Epoch 1/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.3542 - loss: 0.2929
Epoch 2/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.5631 - loss: 0.2066
Epoch 3/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6120 - loss: 0.1860
Epoch 4/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6380 - loss: 0.1749
Epoch 5/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6586 - loss: 0.1654
Epoch 6/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6806 - loss: 0.1562
Epoch 7/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.6955 - loss: 0.1508
Epoch 8/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7080 - loss: 0.1451
Epoch 9/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step - accuracy: 0.7199 - loss: 0.1404
Epoch 10/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7322 - loss: 0.1350
Epoch 11/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.7445 - loss: 0.1309
Epoch 12/25
391/391 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step

In [38]:
# save the model
model.save("cifar-10.h5")

In [45]:
# accuracy on test batch
accuracy_score(np.argmax(y_test,axis=1),np.argmax(model.predict(X_test),axis=1))

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


0.6378

## Testing the model ##

In [40]:
from tkinter import ttk
from tkinter import *
from PIL import Image, ImageTk

labels = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]

def get_rand_prediction():
    random_image_index = np.random.randint(9000)
    
    # Get the image from the test dataset
    img_array = X_test[random_image_index]
    
    # Convert the image array to uint8 (expected by PIL)
    img_array_uint8 = (img_array * 255).astype(np.uint8)
    
    # Convert the image array to a PIL Image
    image_pil = Image.fromarray(img_array_uint8)
    
    image_pil = image_pil.resize((250, 250))
    
    # Convert the PIL image to a format Tkinter can use
    tk_image = ImageTk.PhotoImage(image_pil)
    
    # Update the Tkinter label with the new image
    image_preview.configure(image=tk_image)
    image_preview.image = tk_image  # Keep a reference to avoid garbage collection
    
    # Update the prediction labels
    img_batch = np.expand_dims(X_test[random_image_index], axis=0)  # Prepare the image as a batch (shape: (1, height, width, channels))
    prediction = model.predict(img_batch)  # Predict only for the single image    
    predicted_label.configure(text="Predicted: " + labels[(np.argmax(prediction))].capitalize())
    actual_label.configure(text="Actual: " + labels[(np.argmax(y_test[random_image_index]))].capitalize())

window = Tk()
window.title("CIFAR-10")
window.geometry("500x500")

# Labels for displaying predicted and actual labels
predicted_label = Label(window, text="Predicted:", font=("courrier", 20), fg="brown")
predicted_label.grid(row=0, column=0, padx=5, pady=10, sticky="ew")

actual_label = Label(window, text="Actual:", font=("courrier", 20), fg="green")
actual_label.grid(row=0, column=1, padx=5, pady=10, sticky="ew")

# Label for displaying the image preview
image_preview = Label(window)
image_preview.grid(row=1, column=0, columnspan=2, sticky="nsew")

# Button to get a random image prediction
get_rand_image_button = ttk.Button(window, text="Get Random Image", command=get_rand_prediction)
get_rand_image_button.grid(row=2, column=0, columnspan=2, pady=10, sticky="ns")

window.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
